# SI 630: Homework 2: Word2Vec

This homework will have you implementing word2vec using PyTorch and let you familiarize yourself with building more complex neural networks and the larger PyTorch development infrastructure.

Broadly, this homework consists of a few major parts:
1. Implement a `Corpus` class that will load the dataset and convert it to a sequence of token ids
2. Implement negative sampling to select tokens to be used as negative examples of words in the context
3. Create your dataset of positive and negative examples per context and load it into PyTorch's `DataLoader` to use for sampling
4. Implement a `Word2Vec` class that is a PyTorch neural network
5. Implement a training loop that samples a _batch_ of target words and their respective positive/negative context words
6. Implement rare word removal and frequent word subsampling
7. Run your model on the full dataset for at least one epoch
8. Do the exploratory parts of the homework
9. Make a copy of this notebook and change your implementation so it learns word vectors with less bias

After Step 5, you should be able to run your word2vec implementation on a small dataset and verify that it's learning correctly. Once you can verify everything is working, proceed with steps 6 and beyond. **Please note that this list is a general sketch and the homework PDF has the full list/description of to-dos and all your deliverables.**

In [1]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import init
from tqdm.auto import tqdm, trange
from collections import Counter
import random
from torch import optim

from torch.utils.tensorboard import SummaryWriter

# Helpful for computing cosine similarity--Note that this is NOT a similarity!
from scipy.spatial.distance import cosine

# Handy command-line argument parsing
import argparse

# Sort of smart tokenization
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import MWETokenizer

# We'll use this to save our models
from gensim.models import KeyedVectors

#
# IMPORTANT NOTE: Always set your random seeds when dealing with stochastic
# algorithms as it lets your bugs be reproducible and (more importantly) it lets
# your results be reproducible by others.
#
random.seed(1234)
np.random.seed(1234)
torch.manual_seed(1234)

## Create a class to hold the data

Before we get to training word2vec, we'll need to process the corpus into some representation. The `Corpus` class will handle much of the functionality for corpus reading and keeping track of which word types belong to which ids. The `Corpus` class will also handle the crucial functionality of generating negative samples for training (i.e., randomly-sampled words that were not in the target word's context).

Some parts of this class can be completed after you've gotten word2vec up and running, so see the notes below and the details in the homework PDF.

In [2]:
class Corpus:
    
    def __init__(self):

        self.tokenizer = RegexpTokenizer(r'\w+')
        
        # These state variables become populated with function calls
        #
        # 1. load_data()
        # 2. generate_negative_sampling_table()
        #
        # See those functions for how the various values get filled in

        self.word_to_index = {} # word to unique-id
        self.index_to_word = {} # unique-id to word

        # How many times each word occurs in our data after filtering
        self.word_counts = Counter()

        # A utility data structure that lets us quickly sample "negative"
        # instances in a context. This table contains unique-ids
        self.negative_sampling_table = []
        
        # The dataset we'll use for training, as a sequence of unqiue word
        # ids. This is the sequence across all documents after tokens have been
        # randomly subsampled by the word2vec preprocessing step
        self.full_token_sequence_as_ids = None
        
    def tokenize(self, text):
        '''
        Tokenize the document and returns a list of the tokens
        '''
        
        return self.tokenizer.tokenize(text)        

    def load_data(self, file_name, min_token_freq):
        '''
        Reads the data from the specified file as long long sequence of text
        (ignoring line breaks) and populates the data structures of this
        word2vec object.
        '''

        # Step 1: Read in the file and create a long sequence of tokens for
        # all tokens in the file
        tokenizer2 = MWETokenizer()
        f=open('bio-mwes.txt','r')
        lines=f.readlines()
        for line in lines:
            arr=line.split(' ')
            arr[-1]=arr[-1][:-1]
#             print(arr)
            tokenizer2.add_mwe(tuple(arr))
        f.close()
        
        all_tokens = []
        print('Reading data and tokenizing')
        
        tokenizer = RegexpTokenizer('\w+|\$[\d\.]+')
        fp = open(file_name, 'r')
        lines=fp.readlines()
        for line in lines:
            tokens=tokenizer.tokenize(line)
            tokens=tokenizer2.tokenize(tokens)
#             print(tokens)
            tokens=[i.lower() for i in tokens]
            all_tokens+=tokens
#         print(all_tokens)
    
        # Step 2: Count how many tokens we have of each type
        print('Counting token frequencies')
        
        all_tokens_dict=dict(Counter(all_tokens))
        

        # Step 3: Replace all tokens below the specified frequency with an <UNK>
        # token. 
        #
        # NOTE: You can do this step later if needed
        print("Performing minimum thresholding")
        
        delete_keys1=set()
        all_tokens_dict['<UNK>']=0
        for k,v in all_tokens_dict.items():
            if v<min_token_freq and k!='<UNK>':
                all_tokens_dict['<UNK>']+=1
                delete_keys1.add(k)
        
        for k in delete_keys1:
            del all_tokens_dict[k]
                
        
        

        # Step 4: update self.word_counts to be the number of times each word
        # occurs (including <UNK>)
        self.word_counts=all_tokens_dict
        
        
        # Step 5: Create the mappings from word to unique integer ID and the
        # reverse mapping.
        idx=0
        for k in all_tokens_dict.keys():
            self.word_to_index[k]=idx
            self.index_to_word[idx]=k
            idx+=1
        
        
        # Step 6: Compute the probability of keeping any particular *token* of a
        # word in the training sequence, which we'll use to subsample. This subsampling
        # avoids having the training data be filled with many overly common words
        # as positive examples in the context
        length=len(all_tokens)
        token_set=set()
        delete_keys=set()
        for k,v in all_tokens_dict.items():
            p=v/length
            keep=(np.sqrt(p/0.001)+1)*0.001/(np.sqrt(p))
#             if k=='the' or k=='there' or k=='monday':
#                 print(k,keep)
#             if keep>=0.0358:
            if keep>=random.random()/50:
                token_set.add(k)
                
            
#         for k in delete_keys:
#             print(k)
#             if k in all_tokens_dict.keys():
#                 del all_tokens_dict[k]
#                 del self.index_to_word[self.word_to_index[k]]
#                 del self.word_to_index[k]
            
            
        print("============")
                        
        # Step 7: process the list of tokens (after min-freq filtering) to fill
        # a new list self.full_token_sequence_as_ids where 
        #
        # (1) we probabilistically choose whether to keep each *token* based on the
        # subsampling probabilities (note that this does not mean we drop
        # an entire word!) and 
        #
        # (2) all tokens are convered to their unique ids for faster training.
        #
        # NOTE: You can skip the subsampling part and just do step 2 to get
        # your model up and running.
            
        # NOTE 2: You will perform token-based subsampling based on the probabilities in
        # word_to_sample_prob. When subsampling, you are modifying the sequence itself 
        # (like deleting an item in a list). This action effectively makes the context
        # window  larger for some target words by removing context words that are common
        # from a particular context before the training occurs (which then would now include
        # other words that were previously just outside the window).

        
        unk_id=self.word_to_index['<UNK>']
        self.full_token_sequence_as_ids=[]
        for token in all_tokens:
            if token in token_set:
                self.full_token_sequence_as_ids.append(self.word_to_index[token])
            else:
                self.full_token_sequence_as_ids.append(unk_id)
            
        

        # Helpful print statement to verify what you've loaded
        print('Loaded all data from %s; saw %d tokens (%d unique)' \
              % (file_name, len(self.full_token_sequence_as_ids),
                 len(self.word_to_index)))

        
    def generate_negative_sampling_table(self, exp_power=0.75, table_size=1e6):
        '''
        Generates a big list data structure that we can quickly randomly index into
        in order to select a negative training example (i.e., a word that was
        *not* present in the context). 
        '''       
        
        # Step 1: Figure out how many instances of each word need to go into the
        # negative sampling table. 
        #
        # HINT: np.power and np.fill might be useful here        
        print("Generating sampling table")

        keep_token_dict={}
        total_p=0
        for k in self.index_to_word.keys():
            keep_token_dict[k]=np.power(self.word_counts[self.index_to_word[k]],exp_power)
            total_p+=keep_token_dict[k]
            
        
        for k in self.index_to_word.keys():
            keep_token_dict[k]=keep_token_dict[k]/total_p
        
        
        ct=0
        for idx, val in keep_token_dict.items():
            i=0
            max_val = round(val*table_size)

            while ct < table_size and i <= max_val:
#                 print(idx)
                self.negative_sampling_table.append(idx)
                ct+=1
                i+=1
        
        
        
        # Step 2: Create the table to the correct size. You'll want this to be a
        # numpy array of type int


        # Step 3: Fill the table so that each word has a number of IDs
        # proportionate to its probability of being sampled.
        #
        # Example: if we have 3 words "a" "b" and "c" with probabilites 0.5,
        # 0.33, 0.16 and a table size of 6 then our table would look like this
        # (before converting the words to IDs):
        #
        # [ "a", "a", "a", "b", "b", "c" ]
        #
        


    def generate_negative_samples(self, cur_context_word_id, num_samples):
        '''
        Randomly samples the specified number of negative samples from the lookup
        table and returns this list of IDs as a numpy array. As a performance
        improvement, avoid sampling a negative example that has the same ID as
        the current positive context word.
        '''

        results = []

        # Create a list and sample from the negative_sampling_table to
        # grow the list to num_samples, avoiding adding a negative example that
        # has the same ID as the current context_word
        


        for i in range(num_samples):
            while len(results)!=num_samples:
                idx=random.randint(0,len(self.negative_sampling_table)-1)
                if self.negative_sampling_table[idx] not in cur_context_word_id:
                    results.append(self.negative_sampling_table[idx])
        
#         print(self.index_to_word[cur_context_word_id])
#         for i in results:
#             print(self.index_to_word[i])
        
        return results

## Create the corpus

Now that we have code to turn the text into training data, let's do so. We've provided several files for you to help:

* `wiki-bios.DEBUG.txt` -- use this to debug your corpus reader
* `wiki-bios.10k.txt` -- use this to debug/verify the whole word2vec works
* `wiki-bios.med.txt` -- use this when everything works to generate your vectors for later parts
* `wiki-bios.HUGE.txt.gz` -- _do not use this_ unless (1) everything works and (2) you really want to test/explore. This file is not needed at all to do your homework.

We recommend startin to debug with the first file, as it is small and fast to load (quicker to find bugs). When debugging, we recommend setting the `min_token_freq` argument to 2 so that you can verify that part of the code is working but you still have enough word types left to test the rest.

You'll use the remaining files later, where they're described.

In the next cell, create your `Corpus`, read in the data, and generate the negative sampling table.

In [3]:
corpus = Corpus()
corpus.load_data('wiki-bios.med.txt', 5)
corpus.generate_negative_sampling_table()
# corpus.generate_negative_samples(325,10)

Reading data and tokenizing
Counting token frequencies
Performing minimum thresholding
Loaded all data from wiki-bios.med.txt; saw 22220944 tokens (110331 unique)
Generating sampling table


## Generate the training data

Once we have the corpus ready, we need to generate our training dataset. Each instance in the dataset is a target word and positive and negative examples of contexts words. Given the target word as input, we'll want to predict (or not predict) these positive and negative context words as outputs using our network. Your task here is to create a python `list` of instances. 

Your final training data should be a list of tuples in the format ([target_word_id], [word_id_1, ...], [predicted_labels]), where each item in the list is a list:
1. The first item is a list consisting only of the target word's ID.
2. The second item is a list of word ids for both context words and negative samples 
3. The third item is a list of labels to predicted for each of the word ids in the second list (i.e., `1` for context words and `0` for negative samples). 

You will feed these tuples into the PyTorch `DatasetLoader` later that will do the converstion to `Tensor` objects. You will need to make sure that all of the lists in each tuple are `np.array` instances and are not plain python lists for this `Tensor` converstion to work.

In [4]:
window_size = 2
num_negative_samples_per_target = 2

training_data = []
    
# Loop through each token in the corpus and generate an instance for each, 
# adding it to training_data
ct=0
unk_id=corpus.word_to_index['<UNK>']

for i in tqdm(range(len(corpus.full_token_sequence_as_ids))):
#     print(i)
    idx=corpus.full_token_sequence_as_ids[i]
    
    if idx==unk_id:
        continue
    elif i==0:
        positive_samples_list=corpus.full_token_sequence_as_ids[1:1+window_size]
        negative_samples_num=2*window_size*num_negative_samples_per_target+2
    elif i==1:
        positive_samples_list=corpus.full_token_sequence_as_ids[:1]+corpus.full_token_sequence_as_ids[2:4]
        negative_samples_num=2*window_size*num_negative_samples_per_target+1
    elif i==len(corpus.full_token_sequence_as_ids)-2:
        positive_samples_list=corpus.full_token_sequence_as_ids[i-2:i]+corpus.full_token_sequence_as_ids[i+1:]
        negative_samples_num=2*window_size*num_negative_samples_per_target+1
    elif i==len(corpus.full_token_sequence_as_ids)-1:
        positive_samples_list=corpus.full_token_sequence_as_ids[i-2:i]
        negative_samples_num=2*window_size*num_negative_samples_per_target+2
    else:
        positive_samples_list=[corpus.full_token_sequence_as_ids[i-2],corpus.full_token_sequence_as_ids[i-1],corpus.full_token_sequence_as_ids[i+1],corpus.full_token_sequence_as_ids[i+2]]
        negative_samples_num=2*window_size*num_negative_samples_per_target
    
    
    negative_samples_list=corpus.generate_negative_samples(positive_samples_list, negative_samples_num)
    predicted_labels=[1]*len(positive_samples_list)+[0]*negative_samples_num
    training_data.append((np.array([idx]),np.array(positive_samples_list+negative_samples_list),np.array(predicted_labels)))
    # For exach target word in our dataset, select context words 
    # within +/- the window size in the token sequence
    
    # For each positive target, we need to select negative examples of
    # words that were not in the context. Use the num_negative_samples_per_target
    # hyperparameter to generate these, using the generate_negative_samples()
    # method from the Corpus class

    # NOTE: this part might not make sense until later when you do the training 
    # so feel free to revisit it to see why it happens.
    #
    # Our training will use batches of instances together (compare that 
    # with HW1's SGD that used one item at a time). PyTorch will require
    # that all instances in a batches have the same size, which creates an issue
    # for us here since the target wordss at the very beginning or end of the corpus
    # have shorter contexts. 
    # 
    # To work around these edge-cases, we need to ensure that each instance has
    # the same size, which means it needs to have the same number of positive
    # and negative examples. Since we are short on positive examples here (due
    # to the edge of the corpus), we can just add more negative samples.
    #
    # YOUR TASK: determine what is the maximum number of context words (positive
    # and negative) for any instance and then, for instances that have fewer than
    # this number of context words, add in negative examples.
    #
    # NOTE: The maximum is fixed, so you can precompute this outside the loop
    # ahead of time.
    

  0%|          | 0/22220944 [00:00<?, ?it/s]

## Create the network

We'll create a new neural network as a subclass of `nn.Module` like we did in Homework 1. However, _unlike_ the network you built in Homework 1, we do not need to used linear layers to implement word2vec. Instead, we will use PyTorch's `Emedding` class, which maps an index (e.g., a word id in this case) to an embedding. 

Roughly speaking, word2vec's network makes a prediction by computing the dot product of the target word's embedding and a context word's embedding and then passing this dot product through the sigmoid function ($\sigma$) to predict the probability that the context word was actually in the context. The homework write-up has lots of details on how this works. Your `forward()` function will have to implement this computation.

In [30]:
class Word2Vec(nn.Module):
    
    def __init__(self, vocab_size, embedding_size):
        super(Word2Vec, self).__init__()
        
        # Save what state you want and create the embeddings for your
        # target and context words
        self.target_embeddings = nn.Embedding(vocab_size, embedding_size)
        self.context_embeddings = nn.Embedding(vocab_size, embedding_size)
        
        # Once created, let's fill the embeddings with non-zero random
        # numbers. We need to do this to get the training started. 
        #
        # NOTE: Why do this? Think about what happens if all the embeddings
        # are all zeros initially. What would the predictions look like for
        # word2vec with these embeddings and how would the updated work?
        
        self.init_emb(init_range=0.5/vocab_size)
        
    def init_emb(self, init_range):
        
        # Fill your two embeddings with random numbers uniformly sampled
        # between +/- init_range
        
        self.target_embeddings.weight.data.uniform_(-init_range, init_range)
        self.context_embeddings.weight.data.uniform_(-init_range,init_range)
        
    def forward(self, target_word_id, context_word_ids):
        ''' 
        Predicts whether each context word was actually in the context of the target word.
        The input is a tensor with a single target word's id and a tensor containing each
        of the context words' ids (this includes both positive and negative examples).
        '''
        
        v_t=self.target_embeddings(target_word_id)
        v_c=self.context_embeddings(context_word_ids)

        hid=torch.matmul(v_t,torch.transpose(v_c,2,1))
        out=torch.sigmoid(hid)
        return out
        
        # NOTE 1: This is probably the hardest part of the homework, so you'll
        # need to figure out how to do the dot-product between embeddings and return
        # the sigmoid. Be prepared for lots of debugging. For some reference,
        # our implementation is three lines and really the hard part is just
        # the last line. However, it's usually a matter of figuring out what
        # that one line looks like that ends up being the hard part.
        
        # NOTE 2: In this homework you'll be dealing with *batches* of instances
        # rather than a single instance at once. PyTorch mostly handles this
        # seamlessly under the hood for you (which is very nice) but batching
        # can show in weird ways and create challenges in debugging initially.
        # For one, your inputs will get an extra dimension. So, for example,
        # if you have a batch size of 4, your input for target_word_id will
        # really be 4 x 1. If you get the embeddings of those targets,
        # it then becomes 4x50! The same applies to the context_word_ids, except
        # that was alreayd a list so now you have things with shape 
        #
        #    (batch x context_words x embedding_size)
        #
        # One of your tasks will be to figure out how to get things lined up
        # so everything "just works". When it does, the code looks surprisingly
        # simple, but it might take a lot of debugging (or not!) to get there.
        
        # NOTE 3: We *strongly* discourage you from looking for existing 
        # implementations of word2vec online. Sadly, having reviewed most of the
        # highly-visible ones, they are actually wrong (wow!) or are doing
        # inefficient things like computing the full softmax instead of doing
        # the negative sampling. Looking at these will likely leave you more
        # confused than if you just tried to figure it out yourself.
        
        # NOTE 4: There many ways to implement this, some more efficient
        # than others. You will want to get it working first and then
        # test the timing to see how long it takes. As long as the
        # code works (vector comparisons look good) you'll receive full
        # credit. However, very slow implementations may take hours(!)
        # to converge so plan ahead.
        
        
        # Hint 1: You may want to review the mathematical operations on how
        # to compute the dot product to see how to do these
        
        # Hint 2: the "dim" argument for some operations may come in handy,
        # depending on your implementation
        
        # Hint 3: printing the shape of the tensors can come in very handy when
        # debugging to see where things aren't lining up
           
        # TODO: Implement the forward pass of word2vec
        
    

## Train the network!

Now that you have data in the right format and a neural network designed, it's time to train the network and see if it's all working. The trainin code will look surprisingly similar at times to your pytorch code from Homework 1 since all networks share the same base training setup. However, we'll add a few new elements to get you familiar with more common training techniques. 

For all steps, be sure to use the hyperparameters values described in the write-up.

1. Initialize your optimizer and loss function 
2. Create your network
3. Load your dataset into PyTorch's `DataLoader` class, which will take care of batching and shuffling for us (yay!)
4. Create a new `SummaryWriter` to periodically write our running-sum of the loss to a tensorboard
5. Train your model 

Two new elements show up. First, we'll be using `DataLoader` which is going to sample data for us and put it in a batch (and also convert the data to `Tensor` objects. You can iterate over the batches and each iteration will return all the items eventually, one batch at a time (a full epoch's worth).

The second new part is using `tensorboard`. As you might have noticed in Homework 1, training neural models can take some time. [TensorBoard](https://www.tensorflow.org/tensorboard/) is a handy web-based view that you can check during training to see how the model is doing. We'll use it here and periodically log a running sum of the loss after a set number of steps. The Homework write up has a plot of what this looks like. We'll be doing something simple here with tensorboard but it will come in handy later as you train larger models (for longer) and may want to visually check if your model is converging. TensorBoard was initially written for another deep learning framework, TensorFlow, but proved so useful it was ported to work in PyTorch too and is [easy to integrate](https://pytorch.org/tutorials/recipes/recipes/tensorboard_with_pytorch.html).

To start training, we recommend training on the `wiki-bios.10k.txt` dataset. This data is small enough you can get through an epoch in a few minutes (or less) while still being large enough you can test whether the model is learning anything by examining common words. Below this cell we've added a few helper functions that you can use to debug and query your model. In particular, the `get_neighbors()` function is a great way to test: if your model has learned anything, the nearest neighbors for common words should seem reasonable (without having to jump through mental hoops). An easy word to test on the `10k` data is "january" which should return month-related words as being most similar.

**NOTE**: Since we're training biographies, the text itself will be skewed towards words likely to show up biographices--which isn't necessary like "regular" text. You may find that your model has few instances of words you think are common, or that the model learns poor or unusual neighbors for these. When querying the neighbors, it can help to think of which words you think are likely to show up in biographies on Wikipedia and use those as probes to see what the model has learned.

Once you're convinced the model is learning, switch to the `med` data and train your model as specified in the PDF. Once trained, save your model using the `save()` function at the end of the notebook. This function records your data in a common format for word2vec vectors and lets you load the vectors into other libraries that have more advanced functionality. In particular, you can use the [gensim](https://radimrehurek.com/gensim/auto_examples/tutorials/run_word2vec.html) code in other notebook included to explore the vectors and do simple vector analogies.

In [31]:
def get_neighbors(model, word_to_index, target_word):
    """ 
    Finds the top 10 most similar words to a target word
    """
    outputs = []
    for word, index in tqdm(word_to_index.items(), total=len(word_to_index)):
        similarity = compute_cosine_similarity(model, word_to_index, target_word, word)
        result = {"word": word, "score": similarity}
        outputs.append(result)

    # Sort by highest scores
    neighbors = sorted(outputs, key=lambda o: o['score'], reverse=True)
    return neighbors[1:1000]

def compute_cosine_similarity(model, word_to_index, word_one, word_two):
    '''
    Computes the cosine similarity between the two words
    '''
    try:
        word_one_index = word_to_index[word_one]
        word_two_index = word_to_index[word_two]
    except KeyError:
#         print("============")
#         print(word_one,word_two)
        return 0

#     print(word_one_index,word_two_index)
    embedding_one = model.target_embeddings(torch.LongTensor([word_one_index]))
    embedding_two = model.target_embeddings(torch.LongTensor([word_two_index]))
#     print(embedding_one,embedding_two,word_one_index,word_two_index)
    similarity = 1 - abs(float(cosine(embedding_one.detach().numpy(),
                                      embedding_two.detach().numpy())))
    return similarity

### Reduce bias

In [32]:
def bias_evaluation(model):
    man_woman_sim = (compute_cosine_similarity(model, corpus.word_to_index,"man", "woman")\
        +compute_cosine_similarity(model, corpus.word_to_index,"he", "she"))/2
    er_ress_sim = compute_cosine_similarity(model, corpus.word_to_index,"actor", "actress")\
                +compute_cosine_similarity(model, corpus.word_to_index,"actor", "actress")\
        +compute_cosine_similarity(model, corpus.word_to_index,"waiter", "waitress")\
        +compute_cosine_similarity(model, corpus.word_to_index,"actor", "actress")\
        +compute_cosine_similarity(model, corpus.word_to_index,"steward", "stewardess")
    er_ress_sim=er_ress_sim/5
    return 1 - max(max(man_woman_sim, 0),er_ress_sim)


In [33]:
batch_size = 32
min_token_freq = 5
epochs = 1
embedding_size = 50
learning_rate=5e-5
loss_func = nn.CrossEntropyLoss()
model = Word2Vec(len(corpus.word_counts), embedding_size)
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)

train_loader = DataLoader(dataset=training_data, batch_size=batch_size, shuffle=True)
writer = SummaryWriter()
# HINT: wrapping the epoch/step loops in nested tqdm calls is a great way
# to keep track of how fast things are and how much longer training will take

min_loss = 2000
patience =30
trigger_times = 0
stop_model=False

model.train()

for epoch in range(epochs):

    loss_sum = 0.0
    
    # TODO: use your DataLoader to iterate over the data
    for step, data in enumerate(tqdm(train_loader)):
        
        if stop_model:
            break
        
        # NOTE: since you created the data as a tuple of three np.array instances,
        # these have now been converted to Tensor objects for us
        target_ids, context_ids, labels = data    
        # TODO: Fill in all the training details here
        
        # TODO: Based on the details in the Homework PDF, periodically
        # report the running-sum of the loss to tensorboard. Be sure
        # to reset the running sum after reporting it.
        
        # TODO: it can be helpful to add some early stopping here after
        # a fixed number of steps (e.g., if step > max_steps)
        optimizer.zero_grad()
        y = model(target_ids, context_ids).view(len(target_ids), -1)
        
        # bias evaulation every 20 steps
        if step%20==0:
            loss = loss_func(y, labels.float())+bias_evaluation(model)
        else:
            loss = loss_func(y, labels.float())
        
#         loss = loss_func(y, labels.float())+bias_evaluation(model)
        loss_sum += loss.item()
        loss.backward()
        optimizer.step()
        if step % 100 == 99:
            writer.add_scalar("Loss/train", loss_sum, (step+1)/100)
            print("step: {}, loss: {}".format((step+1)/100, loss_sum))

            # Early Stopping
#             if loss_sum > min_loss:
#                 trigger_times += 1
#                 print('trigger times:', trigger_times)
#                 if trigger_times >= patience:
#                     print('Early stopping!\nStart to test process.')
#                     writer.close()
#                     model.eval()
#                     stop_model=True
#             else:
#                 print('trigger times: 0')
#                 trigger_times = 0
#             min_loss = min(loss_sum,min_loss)
            loss_sum = 0
        
        
        

if not stop_model:          
    writer.close()
# once you finish training, it's good practice to switch to eval.
    model.eval()

  0%|          | 0/680814 [00:00<?, ?it/s]

step: 1.0, loss: 998.1401433944702
step: 2.0, loss: 995.9329824447632
step: 3.0, loss: 994.2385625839233
step: 4.0, loss: 994.141167640686
step: 5.0, loss: 994.3312759399414
step: 6.0, loss: 994.2990665435791
step: 7.0, loss: 994.0350112915039
step: 8.0, loss: 993.7798357009888
step: 9.0, loss: 993.7109670639038
step: 10.0, loss: 993.6447715759277
step: 11.0, loss: 993.6056575775146
step: 12.0, loss: 993.4813098907471
step: 13.0, loss: 993.3919477462769
step: 14.0, loss: 993.2496223449707
step: 15.0, loss: 993.1263790130615
step: 16.0, loss: 993.0352048873901
step: 17.0, loss: 992.9101400375366
step: 18.0, loss: 992.8140020370483
step: 19.0, loss: 992.6486930847168
step: 20.0, loss: 992.4874820709229
step: 21.0, loss: 992.3602924346924
step: 22.0, loss: 992.2697677612305
step: 23.0, loss: 991.9124612808228
step: 24.0, loss: 991.766923904419
step: 25.0, loss: 991.7572374343872
step: 26.0, loss: 991.5710077285767
step: 27.0, loss: 991.5125408172607
step: 28.0, loss: 991.2476806640625
ste

step: 226.0, loss: 969.3612031936646
step: 227.0, loss: 969.143964767456
step: 228.0, loss: 970.1375675201416
step: 229.0, loss: 968.9046154022217
step: 230.0, loss: 969.6152925491333
step: 231.0, loss: 969.399938583374
step: 232.0, loss: 970.3990602493286
step: 233.0, loss: 969.4069957733154
step: 234.0, loss: 969.5409822463989
step: 235.0, loss: 969.4139881134033
step: 236.0, loss: 968.5234308242798
step: 237.0, loss: 968.6493577957153
step: 238.0, loss: 969.040020942688
step: 239.0, loss: 969.0174369812012
step: 240.0, loss: 967.5755271911621
step: 241.0, loss: 969.2416944503784
step: 242.0, loss: 969.0648279190063
step: 243.0, loss: 969.2906551361084
step: 244.0, loss: 968.1310768127441
step: 245.0, loss: 968.3900203704834
step: 246.0, loss: 969.4932661056519
step: 247.0, loss: 968.4427671432495
step: 248.0, loss: 968.566596031189
step: 249.0, loss: 969.6232147216797
step: 250.0, loss: 967.8563852310181
step: 251.0, loss: 968.6176233291626
step: 252.0, loss: 968.3653335571289
step:

step: 449.0, loss: 960.0613126754761
step: 450.0, loss: 959.0369472503662
step: 451.0, loss: 960.4486694335938
step: 452.0, loss: 960.1271924972534
step: 453.0, loss: 959.3364210128784
step: 454.0, loss: 959.1240272521973
step: 455.0, loss: 960.0269594192505
step: 456.0, loss: 958.7825269699097
step: 457.0, loss: 959.3619508743286
step: 458.0, loss: 959.5500965118408
step: 459.0, loss: 958.859787940979
step: 460.0, loss: 959.4976625442505
step: 461.0, loss: 959.4323167800903
step: 462.0, loss: 958.0067157745361
step: 463.0, loss: 959.1380434036255
step: 464.0, loss: 958.8501987457275
step: 465.0, loss: 958.3387250900269
step: 466.0, loss: 957.5407495498657
step: 467.0, loss: 957.7642126083374
step: 468.0, loss: 958.4733276367188
step: 469.0, loss: 959.3065214157104
step: 470.0, loss: 958.5182466506958
step: 471.0, loss: 959.113317489624
step: 472.0, loss: 958.7238903045654
step: 473.0, loss: 957.6771326065063
step: 474.0, loss: 958.5432615280151
step: 475.0, loss: 959.1093521118164
ste

step: 672.0, loss: 953.6053810119629
step: 673.0, loss: 953.3173770904541
step: 674.0, loss: 952.8842344284058
step: 675.0, loss: 952.2291851043701
step: 676.0, loss: 954.2591648101807
step: 677.0, loss: 952.596396446228
step: 678.0, loss: 954.2597513198853
step: 679.0, loss: 954.2100057601929
step: 680.0, loss: 953.2261009216309
step: 681.0, loss: 953.0515537261963
step: 682.0, loss: 953.4767055511475
step: 683.0, loss: 954.3754634857178
step: 684.0, loss: 952.8189611434937
step: 685.0, loss: 951.5963134765625
step: 686.0, loss: 953.8276510238647
step: 687.0, loss: 953.513671875
step: 688.0, loss: 953.254225730896
step: 689.0, loss: 954.1397876739502
step: 690.0, loss: 953.2795391082764
step: 691.0, loss: 952.9120216369629
step: 692.0, loss: 952.6092767715454
step: 693.0, loss: 952.8358097076416
step: 694.0, loss: 954.0168190002441
step: 695.0, loss: 952.851803779602
step: 696.0, loss: 953.1328439712524
step: 697.0, loss: 953.9074544906616
step: 698.0, loss: 953.5862712860107
step: 69

...

step: 5889.0, loss: 925.9449119567871
step: 5890.0, loss: 925.8985433578491
step: 5891.0, loss: 925.9684247970581
step: 5892.0, loss: 926.9565267562866
step: 5893.0, loss: 926.6530513763428
step: 5894.0, loss: 925.8254880905151
step: 5895.0, loss: 927.8571996688843
step: 5896.0, loss: 928.1449136734009
step: 5897.0, loss: 926.7371501922607
step: 5898.0, loss: 925.8038911819458
step: 5899.0, loss: 926.5033235549927
step: 5900.0, loss: 924.5322494506836
step: 5901.0, loss: 925.3994159698486
step: 5902.0, loss: 928.2102928161621
step: 5903.0, loss: 927.1822481155396
step: 5904.0, loss: 927.9933423995972
step: 5905.0, loss: 926.4492111206055
step: 5906.0, loss: 926.3422899246216
step: 5907.0, loss: 928.3331184387207
step: 5908.0, loss: 926.3933343887329
step: 5909.0, loss: 925.3531999588013
step: 5910.0, loss: 926.1817007064819
step: 5911.0, loss: 926.1704530715942
step: 5912.0, loss: 925.564190864563
step: 5913.0, loss: 925.1602792739868
step: 5914.0, loss: 924.8855648040771
step: 5915.0,

step: 6106.0, loss: 926.6923198699951
step: 6107.0, loss: 927.7827730178833
step: 6108.0, loss: 927.0943355560303
step: 6109.0, loss: 927.1796455383301
step: 6110.0, loss: 927.0687828063965
step: 6111.0, loss: 926.4156217575073
step: 6112.0, loss: 925.0986938476562
step: 6113.0, loss: 926.5244874954224
step: 6114.0, loss: 925.6469278335571
step: 6115.0, loss: 924.581051826477
step: 6116.0, loss: 925.0303106307983
step: 6117.0, loss: 924.3282880783081
step: 6118.0, loss: 925.6087331771851
step: 6119.0, loss: 926.0888071060181
step: 6120.0, loss: 925.7141246795654
step: 6121.0, loss: 924.1774396896362
step: 6122.0, loss: 924.9632215499878
step: 6123.0, loss: 925.9814739227295
step: 6124.0, loss: 924.3520240783691
step: 6125.0, loss: 926.4185247421265
step: 6126.0, loss: 926.5818948745728
step: 6127.0, loss: 927.9224824905396
step: 6128.0, loss: 925.0171766281128
step: 6129.0, loss: 924.1602010726929
step: 6130.0, loss: 925.1161737442017
step: 6131.0, loss: 924.9676284790039
step: 6132.0,

step: 6323.0, loss: 924.8445062637329
step: 6324.0, loss: 924.0101919174194
step: 6325.0, loss: 925.3958072662354
step: 6326.0, loss: 926.2946634292603
step: 6327.0, loss: 925.8225975036621
step: 6328.0, loss: 925.472785949707
step: 6329.0, loss: 923.4855241775513
step: 6330.0, loss: 926.9584321975708
step: 6331.0, loss: 924.6883115768433
step: 6332.0, loss: 926.0530405044556
step: 6333.0, loss: 925.0236501693726
step: 6334.0, loss: 926.2546768188477
step: 6335.0, loss: 925.1619253158569
step: 6336.0, loss: 924.6172552108765
step: 6337.0, loss: 926.9287233352661
step: 6338.0, loss: 923.2123613357544
step: 6339.0, loss: 924.8553447723389
step: 6340.0, loss: 924.662483215332
step: 6341.0, loss: 926.1748561859131
step: 6342.0, loss: 927.0042734146118
step: 6343.0, loss: 927.3249959945679
step: 6344.0, loss: 923.5956106185913
step: 6345.0, loss: 925.5728988647461
step: 6346.0, loss: 927.9524259567261
step: 6347.0, loss: 926.0359382629395
step: 6348.0, loss: 925.8848218917847
step: 6349.0, 

step: 6540.0, loss: 923.6131496429443
step: 6541.0, loss: 924.2903347015381
step: 6542.0, loss: 923.9341106414795
step: 6543.0, loss: 923.8497905731201
step: 6544.0, loss: 924.2965421676636
step: 6545.0, loss: 926.9124412536621
step: 6546.0, loss: 924.3603363037109
step: 6547.0, loss: 924.863881111145
step: 6548.0, loss: 925.25794506073
step: 6549.0, loss: 924.8901948928833
step: 6550.0, loss: 922.8823041915894
step: 6551.0, loss: 925.2858572006226
step: 6552.0, loss: 926.4309740066528
step: 6553.0, loss: 925.4223041534424
step: 6554.0, loss: 924.252537727356
step: 6555.0, loss: 924.1106719970703
step: 6556.0, loss: 926.1634435653687
step: 6557.0, loss: 925.482494354248
step: 6558.0, loss: 926.230749130249
step: 6559.0, loss: 924.9860992431641
step: 6560.0, loss: 924.9309206008911
step: 6561.0, loss: 924.4993762969971
step: 6562.0, loss: 925.7169933319092
step: 6563.0, loss: 925.8278312683105
step: 6564.0, loss: 925.582986831665
step: 6565.0, loss: 925.0573329925537
step: 6566.0, loss:

step: 6757.0, loss: 925.7204103469849
step: 6758.0, loss: 924.2942247390747
step: 6759.0, loss: 924.9566287994385
step: 6760.0, loss: 924.173921585083
step: 6761.0, loss: 924.9002656936646
step: 6762.0, loss: 925.0846967697144
step: 6763.0, loss: 925.2010707855225
step: 6764.0, loss: 924.8735752105713
step: 6765.0, loss: 925.413984298706
step: 6766.0, loss: 924.5813302993774
step: 6767.0, loss: 926.2609348297119
step: 6768.0, loss: 922.7355518341064
step: 6769.0, loss: 927.044979095459
step: 6770.0, loss: 926.0031261444092
step: 6771.0, loss: 924.3677520751953
step: 6772.0, loss: 925.2375450134277
step: 6773.0, loss: 924.8982877731323
step: 6774.0, loss: 925.1755485534668
step: 6775.0, loss: 923.8886451721191
step: 6776.0, loss: 926.6180381774902
step: 6777.0, loss: 925.1117076873779
step: 6778.0, loss: 926.1805448532104
step: 6779.0, loss: 924.3604021072388
step: 6780.0, loss: 925.2612199783325
step: 6781.0, loss: 923.9717206954956
step: 6782.0, loss: 924.0715637207031
step: 6783.0, l

## Verify things are working

Once you have an initial model trained, try using the following code to query the model for what are the nearest neighbor of a word. This code is intended to help you debug

In [35]:
get_neighbors(model, corpus.word_to_index, "january")

  0%|          | 0/110331 [00:00<?, ?it/s]

[{'word': 'june', 'score': 0.999844491481781},
 {'word': 'february', 'score': 0.9997945427894592},
 {'word': 'november', 'score': 0.9997267723083496},
 {'word': 'july', 'score': 0.9996899962425232},
 {'word': 'march', 'score': 0.9996562004089355},
 {'word': 'september', 'score': 0.9995689392089844},
 {'word': 'august', 'score': 0.9995659589767456},
 {'word': 'october', 'score': 0.999500036239624},
 {'word': 'april', 'score': 0.9994309544563293},
 {'word': 'december', 'score': 0.9990468621253967},
 {'word': '30', 'score': 0.9873328804969788},
 {'word': 'aged', 'score': 0.9869542121887207},
 {'word': '28', 'score': 0.9868108034133911},
 {'word': '26', 'score': 0.9867387413978577},
 {'word': 'windsor_park', 'score': 0.9863975048065186},
 {'word': 'dec', 'score': 0.9863268733024597},
 {'word': 'pandemic', 'score': 0.9861127734184265},
 {'word': '86', 'score': 0.9848241806030273},
 {'word': '27', 'score': 0.9846280217170715},
 {'word': '84', 'score': 0.9844803810119629},
 {'word': '91', 'sc

# Save your model!

Once you have a fully trained model, save it using the code below. Note that we only save the `target_embeddings` from the model, but you could modify the code if you want to save the context vectors--or even try doing fancier things like saving the concatenation of the two or the average of the two!

In [36]:
def save(model, corpus, filename):
    '''
    Saves the model to the specified filename as a gensim KeyedVectors in the
    text format so you can load it separately.
    '''

    # Creates an empty KeyedVectors with our embedding size
#     kv = KeyedVectors(vector_size=model.embedding_size) 
    kv=KeyedVectors(vector_size=50)
    vectors = []
    words = []
    # Get the list of words/vectors in a consistent order
    for index in trange(model.target_embeddings.num_embeddings):
        word = corpus.index_to_word[index]
        vectors.append(model.target_embeddings(torch.LongTensor([index])).detach().numpy()[0])
        words.append(word)

    # Fills the KV object with our data in the right order
    kv.add_vectors(words, vectors) 
    kv.save_word2vec_format(filename, binary=False)


In [ ]:
model.target_embeddings.num_embeddings

In [37]:
save(model,corpus,'word2vec_batch32_debias_med_2.kv')

  0%|          | 0/110331 [00:00<?, ?it/s]

# FINAL PART: DO THIS LAST AND READ CAREFULLY

Before you start this part, you need to have a fully working solution and completed the exploratory part of the assignment.

**Once you are ready, create a copy of your working notebook and call it `Debiased Word2Vec.ipynb`. Do not do this part in your working code for the assignment!!!**

## Seriously, save your code in a new file and then start reading the rest of these instructions there.

Ok, hopefully you're reading these in a new file... For this last part of the assignment, we're going to _change_ how word2vec learns at a fundamental level. 

As you might have noticed in your exploratory analysis, the word2vec model learns to weird and sometimes biased associations between words. In particular, your word2vec model has likely learned some unfortunate gender biases, e.g., that the vector for "nurse" is closer to "woman" than "man". The algorithm itself isn't to blame since it is learning these from a corpus (here, Wikipedia biographies) that contain biases already based on how people write. Wikipedia [is](http://markusstrohmaier.info/documents/2015_icwsm2015_wikipedia_gender.pdf) [well](http://dcs.gla.ac.uk/~mounia/Papers/wiki_gender_bias.pdf) [known](https://www.academia.edu/download/64856696/genderanalysisofWikipediabiostext_self_archived.pdf) for having gender biases in how it writes about men and women.


**Side note**: Some times this bias-learning behavior is useful: We can use word2vec to uncover these biases and analyze their trends, like this PNAS paper did for [looking at bias in news writing along multiple dimensions of identity](https://www.pnas.org/content/pnas/115/16/E3635.full.pdf)

In this last part of the homework, we'll ask how we might try to _prevent_ these biases by modifying the training. You won't need to solve this problem by any means, but the act of trying to reduce the biases will open up a whole new toolbox for how you (the experimenter/practioner) can change how and what models learn.

There are many potential ways to _debias_ word embeddings so that their representations are not skewed along one "latent dimension" like gender. In this homework, you'll be trying one of a few different ideas for how to do it. **You are not expected to solve gender bias! This part of the assignment is to have to start grappling with a hard challenge but there is no penalty for doing less-well!** 

One common technique to have models avoid learning bias is similar to another one you already&mdash;**regularization**. In Logistic Regression, we could use L2 regularization to have our model avoid learning $\beta$ weights that are overfit to specific or low-frequency features by adding a regularizer penalty where the larger the weight, the more penalty the model paid. Recall that this forces the model to only pick the most useful (generalizable) weights, since it has to pay a penalty for any non-zero weight. 

In word2vec, we can adapt the idea to think about whether our model's embeddings are closer or farther to different gender dimensions. For example, if we consider the embedding for "president", ideally, we'd want it to be equally similar to the embeddings for "man" and "woman". One idea then is to penalize the model based on how uneven the similarity is. We can do this by directly modifying the loss:
```
loss = loss_criteron(preds, actual_vals) + some_bias_measuring_function(model)
```
Here, the `some_bias_measuring_function` function takes in your model as input and returns how much bias you found. Continuing our example, we might implement it in pseudocode as
```
def some_bias_measuring_function(model):
    pres_woman_sim = cosine_similarity(model, "president", "woman")
    pres_man_sim = cosine_similarity(model, "president", "man")
    return abs(pres_woman_sim - pres_man_sim)
```
This simple example would penalize the model for learning a representation of "president" that is more simular to one of the two words. Of course, this example is overly simple. Why just "president"? Why just "man" and "woman"? Why not other words or other gender-related words or other gender identities?? 

Another idea might be to just make the vectors for "man" and "woman" be as similar as possible:
```
def some_bias_measuring_function(model):
    # cosine similarity is in [-1,1] but we mostly expect it in [0,1]
    man_woman_sim = cosine_similarity(model, "man", "woman")
    # penalize vectors that are not maximally similar, and avoid the edge case 
    # of negative cosine similarity
    return 1 - max(man_woman_sim, 0)
```

All of this works in practice because PyTorch is fantastic about tracking the gradient with respect to the loss. This ability lets us easily define a loss function so that our word2vec model (1) learns to predict the right context words while (2) avoids learning biases. If we compare this code to the numpy part of Homework 1, it's easy to see how powerful PyTorch can be as an experimenter for helping you control what and how your models learn!

Your task is to expand this general approach by coming up with an extension to word2vec that adds some new term to the `loss` value that penalizes bias in the gender dimension. There is no right way to do this and even some right-looking approaches may not work&mdash;or might word but simultaneously destroy the information in the word vectors (all-zero vectors are unbiased but also uninformative!). 

**Suggestion:** You may need to weight your bias term in the loss function (remember that $\lambda_1 x_1 + \lambda_2 x_2$ interpolation? This is sort of similar) so that your debiasing regularizer doesn't overly penalize your model.

Once you have generated your model, record word vector similarities for the pairs listed on canvas in `word-pair-similarity-predictions.csv` where your file writes a result like
```
word1,word2,sim
dog,puppy,0.91234123
woman,preseident,0.81234
```
You'll record the similarity for each pair of words in the file and upload it to CodaLab, which is kind of like Kaggle but lets use a custom scoring program. We'll evaluate your embeddings based on how unbiased they are and how much information they still capture after debiasing. **Your grade does not depend on how well you do in CodaLab, just that you tried something and submitted.** However, the CodaLab leaderboard will hopefully provide a fun and insightful way of comparing just how much bias we can remove from our embeddings.

The CodaLab link will be posted to Piazza